# Búsqueda semántica
Autor: Eric S. Tellez <eric.tellez@infotec.mx>


# Language models
En los word embeddings, cada palabra del vocabulario tiene asignado un vector denso de manera estática, fruto de su semántica basada en la hipótesis distribucional. Los modelos de lenguaje van más allá, intentando no solo tener en cuenta una palabra para el vector, si no que el vector mismo es dependiente del contexto, por lo cual puede desambiguar de manera natural palabras idénticas (homónimos) usando dicha información contextual. Adicionalmente, capturan información relevante de grandes corpus de texto, aportando muchas veces información de un mundo (i.e., si se entrenan usando la wikipedia, tendrán información relavante de multiples dominios).

La punta de lanza de los modelos de lenguaje es el aprendizaje profundo, y más precisamente, con el uso de _transformers_. Uno de los modelos de lenguaje más utilizado es BERT [@VSPU2017]. La persona interesada más sobre ellos <https://huggingface.co/>. 
Cuando la información esta poco específicada, las palabras adecuadas podrían ser dificiles de tener o limitar. En estos casos, la representaciones semánticas que permiten buscar _lo que se desea_ por medio de conceptos nos acerca más a la posibilidad de obtener información útil.

En lo que resta de este notebook se estarán viendo como usar los modelos, y más precisamente los embeddings para realizar tareas de recuperación de información.


In [1]:
using Pkg
Pkg.activate(".")


  Activating project at `~/Cursos/IR-2024/Unidades`


In [2]:
using SimilaritySearch, LinearAlgebra, HypertextLiteral, CodecZlib, JSON, Random, StatsBase, DataFrames, JLD2
using Downloads: download
using Transformers
using Transformers.TextEncoders
using Transformers.HuggingFace

In [3]:
textencoder, bertmodel = hgf"dccuchile/bert-base-spanish-wwm-cased"

(BertTextEncoder(
├─ TextTokenizer(MatchTokenization(WordPieceTokenization(bert_cased_tokenizer, WordPiece(vocab_size = 31002, unk = [UNK], max_char = 100)), 5 patterns)),
├─ vocab = Vocab{String, SizedArray}(size = 31002, unk = [UNK], unki = 4),
├─ startsym = [CLS],
├─ endsym = [SEP],
├─ padsym = [PAD],
├─ trunc = 512,
└─ process = Pipelines:
  ╰─ target[token] := TextEncodeBase.nestedcall(string_getvalue, source)
  ╰─ target[token] := Transformers.TextEncoders.grouping_sentence(target.token)
  ╰─ target[(token, segment)] := SequenceTemplate{String}([CLS]:<type=1> Input[1]:<type=1> [SEP]:<type=1> (Input[2]:<type=2> [SEP]:<type=2>)...)(target.token)
  ╰─ target[attention_mask] := (NeuralAttentionlib.LengthMask ∘ Transformers.TextEncoders.getlengths(512))(target.token)
  ╰─ target[token] := TextEncodeBase.trunc_and_pad(512, [PAD], tail, tail)(target.token)
  ╰─ target[token] := TextEncodeBase.nested2batch(target.token)
  ╰─ target[segment] := TextEncodeBase.trunc_and_pad(512, 1, tail, t

In [4]:
function encode_text_(bertmodel, e)
    s = bertmodel(e)
    p = vec(s.pooled)
    normalize!(p)
end

function encode_text(textencoder, bertmodel, text::AbstractString)
    e = encode(textencoder, text)
    encode_text_(bertmodel, e)
end

function encode_text(textencoder, bertmodel, text::AbstractVector)
    length(text) == 2 || throw(ArgumentError("the model supports a sentence and a pair of sentences"))
    e = encode(textencoder, [text])
    encode_text_(bertmodel, e)
end

function encode_corpus(textencoder, bertmodel, corpus::AbstractVector)
    # we avoid using the Bert / Beto API directly to reduce memory consumption
    # nonetheless, it will make use of more dynamic allocations
    n = length(corpus)
    E = Matrix{Float32}(undef, 768, n)
    for (i, text) in enumerate(corpus)
        if rand() < 0.01
            @info i, text
        end
        E[:, i] .= encode_text(textencoder, bertmodel, text)
    end

    E
end

function create_index(db, recall)
    dist = NormalizedCosineDistance()
    index = SearchGraph(; dist, db, verbose=false)
    index!(index; callbacks=SearchGraphCallbacks(MinRecall(recall)))
    optimize!(index, MinRecall(recall))
    index
end

create_index (generic function with 1 method)

In [5]:
display(@htl "<h1>Cargando el corpus</h1>")

include("read_datasets.jl")

D, Q = DataFrame.(read_news())

first(D, 10)

<h1>Cargando el corpus</h1>

Row,corpus,labels
,String,String
1,RT @SanGonAzc: @RadioIndeMexico @FiscaliaNL @FGRMexico @lopezobrador_ @AldoFasciZ @RicardoMeb se van a seguir haciendo tontos?? #debaniesc…,AdriDelgadoRuiz
2,"“El periodismo serio y responsable es una herramienta esencial para salvaguardar y fortalecer el sistema democrático en México y el mundo, y hoy se debe fortalecer esa profesión”, afirmó el periodista y escritor, Harris Whitbeck @harriswhitbeck\n#ElDedoEnLaLLaga #HeraldoRadio",AdriDelgadoRuiz
3,"""...Sí esperamos que se aumente la tasa en 25 puntos base en las siguientes dos reuniones y continuar así durante el primer trimestre del 2022”: @raguilar_abe",AdriDelgadoRuiz
4,RT @Univ_Cultura: Encuentran sistema hidráulico de hace 400 años en catedral en Xochimilco https://t.co/ncdhF9Soqf https://t.co/XGOLMTPBfh,El_Universal_Mx
5,"Los tacos son grandes embajadores de la comida mexicana en el mundo. Sin embargo, la reconocida chef Gabriela Cámara asegura que son solo «una parte ínfima» de la gastronomía de su país. https://t.co/i6xhA6PgXA",CNNEE
6,Esta es la lista de países a los que Estados Unidos donará vacunas anticovid https://t.co/hsB5vbWSOc,NTN24
7,"Por desgracia, esta tendencia impactará en la elevada inflación y golpeará los presupuestos de las familias estadounidenses. ¿A qué se debe este aumento?\nhttps://t.co/8OnSFQkmJ3",UniNoticias
8,"La embajadora de Ucrania en Estados Unidos, Oksana Markarova, recibe aplausos de solidaridad durante el discurso del Estado de la Nación de Joe Biden. \n\nSigue lo más destacado de #SOTU22 \n👉🏻https://t.co/6d1JMFcQPx\n\n#SOTU2022 https://t.co/2qSZHfdefh",TelemundoNews
9,"Dos Bocas alista motores | La refinería ""Olmeca"" arranca pruebas en el cuarto aniversario del triunfo de AMLO https://t.co/HEXX2rBsnG https://t.co/qPrqUK77Ps",SinEmbargoMX


In [6]:
trainfile = "spanish-twitter-news-and-opinions-top25-68.train.h5"
testfile = "spanish-twitter-news-and-opinions-top25-68.test.h5"

if !isfile(trainfile)
    @time train = encode_corpus(textencoder, bertmodel, D.corpus)
    jldsave(trainfile; train)
else
    train = jldopen(trainfile) do f
        f["train"]
    end
end

if !isfile(testfile)
    @time test = encode_corpus(textencoder, bertmodel, Q.corpus)
    jldsave(testfile; test)
else
    test = jldopen(testfile) do f
        f["test"]
    end
end

size(train), size(test)

((768, 30244), (768, 3361))

### Se crea el índice métrico

In [7]:
@time index = create_index(MatrixDatabase(train), 0.95)

  8.923241 seconds (4.75 M allocations: 328.891 MiB, 0.70% gc time, 48.71% compilation time)


SearchGraph{NormalizedCosineDistance, MatrixDatabase{Matrix{Float32}}, SimilaritySearch.AdjacencyLists.AdjacencyList{UInt32}, BeamSearch}
  dist: NormalizedCosineDistance NormalizedCosineDistance()
  db: MatrixDatabase{Matrix{Float32}}
  adj: SimilaritySearch.AdjacencyLists.AdjacencyList{UInt32}
  hints: Array{Int32}((94,)) Int32[40, 196, 292, 297, 300, 318, 392, 459, 495, 565  …  2985, 3003, 3005, 3014, 3054, 3057, 3059, 3102, 3104, 3131]
  search_algo: BeamSearch
  len: Base.RefValue{Int64}
  verbose: Bool false


### Búsqueda de todos los vecinos cercanos de test en train, observe la conveniencia del uso de un índice

Entre más sean las consultas más se ve la bondad

In [13]:
let k = 32
    @time Iknns, Idists = searchbatch(index, MatrixDatabase(test), k)
    exs = ExhaustiveSearch(; dist=distance(index), db=database(index))
    @time Eknns, Edists = searchbatch(exs, MatrixDatabase(test), k)
    @show macrorecall(Eknns, Iknns)
end

  0.127557 seconds (6.73 k allocations: 945.688 KiB)
  0.880833 seconds (9 allocations: 840.531 KiB)
macrorecall(Eknns, Iknns) = 0.9515118268372508


0.9515118268372508

### Búsqueda y presentación de los resultados

In [9]:
E = (; textencoder, bertmodel, index, data=D); # para simplificar algunas funciones

In [10]:
function search_and_display(E, qtext, k)
    res = KnnResult(k)
    q = encode_text(E.textencoder, E.bertmodel, qtext)
    @time search(index, q, res)
    
    L = []
    for (j, p) in enumerate(res)
        id, d = p.id, p.weight
        text, label = E.data.corpus[id], E.data.labels[id]
        push!(L, @htl "<tr><td>$j</td><td>$(id)</td><td>$(round(d, digits=3))</td> <td>$(label)</td><td> $(text)</td> </tr>")
    end

    display(@htl """<h2>resultados for "$qtext"</h2>
    <table>
    <th>  <td>id</td> <td>dist</td> <td>user</td> </td>message<td> </th>
        $L
    </table>
    """)
end


search_and_display (generic function with 1 method)

In [11]:

display(@htl "<h1>Ejemplos de búsqueda</h1>")
search_and_display(E, "evento astronomico", 7)
search_and_display(E, "trafico de drogas", 7)
search_and_display(E, "covid corona virus", 7)

<h1>Ejemplos de búsqueda</h1>

  0.165262 seconds (99.59 k allocations: 6.359 MiB, 99.20% compilation time)


1,9839,0.944,PublimetroMX,"Amber Heard declaró que Depp la agredió física y sexualmente en múltiples ocasiones, generalmente cuando supuestamente estaba borracho o drogado. https://t.co/B5JpHfI3o6"
2,18657,0.946,epigmenioibarra,"No se olvida 1968-1971 La demanda estudiantil de libertad y democracia fue reprimida violentamente por los gobiernos de Gustavo Díaz Ordaz y Luis Echeverría. Por años, cuerpos policiacos, militares y paramilitares realizaron persecuciones, desapariciones, tortura y asesinatos. https://t.co/3foPXB2UX2"
3,16060,0.947,epigmenioibarra,"AyotzinapaFueElEstado El ejército sí salió esa noche del cuartel d Iguala. Militares estuvieron en escena del crimen El ejército, único con capacidad operativa,fuerza y medios p operar desaparición masiva #NiPerdonNiOlvido https://t.co/CxRHFYPv4n"
4,21008,0.947,epigmenioibarra,"No se olvida 1968-1971 La demanda estudiantil de libertad y democracia fue reprimida violentamente por los gobiernos de Gustavo Díaz Ordaz y Luis Echeverría. Por años, cuerpos policiacos, militares y paramilitares realizaron persecuciones, desapariciones, tortura y asesinatos. https://t.co/p87Dd4qdY6"
5,12897,0.948,CNNEE,"¿Deberían las personas usar mascarillas en la oficina o lugar de trabajo, incluso si no están obligados a hacerlo? https://t.co/ipPdBB2cS9"
6,14241,0.948,UniNoticias,"🇷🇺 Donbas, el polvorín del este de Ucrania que puede servir de excusa para una invasión rusa. 🇺🇦 En esa zona hay un enfrentamiento entre grupos separatistas prorrusos y fuerzas ucranianas. https://t.co/90nkyG70L9"
7,7150,0.949,CNNEE,"Una explosión de pirotecnia se registró en la zona de Tlalnepantla, en el estado mexicano de Morelos, según informó este miércoles el gobernador de la entidad, Cuauhtémoc Blanco. https://t.co/DrAHthAu7h"


  0.001361 seconds (4 allocations: 96 bytes)


1,14241,0.911,UniNoticias,"🇷🇺 Donbas, el polvorín del este de Ucrania que puede servir de excusa para una invasión rusa. 🇺🇦 En esa zona hay un enfrentamiento entre grupos separatistas prorrusos y fuerzas ucranianas. https://t.co/90nkyG70L9"
2,520,0.914,bbcmundo,"Las autoridades ucranianas han descubierto evidencias dejadas por las tropas rusas en Bucha, donde un campo de verano para niños terminó convertido en un centro de ejecución de civiles. https://t.co/cwJuqwOFLt"
3,16300,0.915,CNNEE,El portavoz del Comando Central de las Fuerzas Armadas describió el ataque como una operación en la que se usó un sofisticado sistema de radar que puede detectar el blanco a gran distancia y las bombas fueron lanzadas desde una nave no tripulada https://t.co/13PPwX4A5N
4,17030,0.915,epigmenioibarra,"Ayotzinapa no fue un hecho aislado, fue parte de un patrón de corrupción y represión. La desaparición de los 43 normalistas en Iguala sintetizó la descomposición del régimen político mexicano. Somos memoria. ¡Empezamos PaseDeLista1al43 ahora! #Ayotzinapa7años https://t.co/JypzRPiFue"
5,17065,0.915,UniNoticias,El tiroteo ocurrió en una estación de autobuses cercana al Pentágono. https://t.co/umnPboh655
6,1971,0.916,SinEmbargoMX,"#FOTOS | En pleno cruce de las calles González y América, en la colonia Hidalgo de esa comuna fronteriza, se produjo el colapso del pavimento y todo el terreno, formando un cráter de 20 metros de diámetro y ocho de profundidad https://t.co/ISC8rvn2Sx https://t.co/jlJz7bnt3y"
7,6271,0.917,epigmenioibarra,"Ayotzinapa no fue un hecho aislado, fue parte de un patrón de corrupción y represión. La desaparición de los 43 normalistas en Iguala sintetizó la descomposición del régimen político mexicano. Somos memoria. ¡Empezamos PaseDeLista1al43 ahora! #Ayotzinapa92Meses https://t.co/3R6iYQ7Km9"


  0.001319 seconds (4 allocations: 96 bytes)


1,21008,0.914,epigmenioibarra,"No se olvida 1968-1971 La demanda estudiantil de libertad y democracia fue reprimida violentamente por los gobiernos de Gustavo Díaz Ordaz y Luis Echeverría. Por años, cuerpos policiacos, militares y paramilitares realizaron persecuciones, desapariciones, tortura y asesinatos. https://t.co/p87Dd4qdY6"
2,18657,0.915,epigmenioibarra,"No se olvida 1968-1971 La demanda estudiantil de libertad y democracia fue reprimida violentamente por los gobiernos de Gustavo Díaz Ordaz y Luis Echeverría. Por años, cuerpos policiacos, militares y paramilitares realizaron persecuciones, desapariciones, tortura y asesinatos. https://t.co/3foPXB2UX2"
3,17229,0.917,epigmenioibarra,"No se olvida 1968-1971 La demanda estudiantil de libertad y democracia fue reprimida violentamente por los gobiernos de Gustavo Díaz Ordaz y Luis Echeverría. Por años, cuerpos policiacos, militares y paramilitares realizaron persecuciones, desapariciones, tortura y asesinatos. https://t.co/9cnGIlRU1d"
4,623,0.918,epigmenioibarra,"Guerra en México 2007 Tras cometer fraude electoral, Felipe Calderón simuló una ""guerra"" al narco y lanzó el ejército a las calles para ""exterminar al enemigo"". Se multiplicaron la violencia y los asesinatos; se fortaleció el crimen organizado. Un baño de sangre cubrió al país. https://t.co/3Wf9CVa4rq"
5,10408,0.918,Radio_Formula,Los legisladores coincidieron que esto contraviene a lo mencionado en la Constitución Mexicana en el principio de igualdad entre hombre y mujer. https://t.co/DgdZtThswD
6,16554,0.918,TelemundoNews,La policía detiene al sospechoso del tiroteo en un salón de belleza de Dallas. El FBI abre investigación por delito de odio. https://t.co/WEkzuEccLx
7,30054,0.918,Radio_Formula,El excandidato presidencial del PAN pudo haber sido profesor de la máxima casa de estudios. https://t.co/r0M3wsf0FF


In [12]:
display(@htl "<h1>Ejemplos de búsqueda (mensajes aleatorios sobre el train)</h1>")

for i in 1:3
    for qid in rand(1:length(E.data.corpus))
        search_and_display(E, E.data.corpus[qid], 7)
    end
end

<h1>Ejemplos de búsqueda (mensajes aleatorios)</h1>

  0.001330 seconds (4 allocations: 96 bytes)


1,9839,0.934,PublimetroMX,"Amber Heard declaró que Depp la agredió física y sexualmente en múltiples ocasiones, generalmente cuando supuestamente estaba borracho o drogado. https://t.co/B5JpHfI3o6"
2,893,0.936,AristeguiOnline,Gobernador de Texas prohíbe los mandatos federales de vacunación contra Covid-19; critica a Biden por 'bullying' https://t.co/aHL0MRes75 https://t.co/60KjluhQzo
3,16684,0.936,PublimetroCol,Elden Ring presentará gameplay el día de mañana https://t.co/SeS9oHtyvK
4,25599,0.937,bbcmundo,"Además, hay que aceptar un diagnóstico psiquiátrico de ""trastorno de identidad de género"", entre otros requisitos. https://t.co/fZfpbam1JD"
5,12515,0.938,el_pais,RT @elpais_america: Los estrenos de ‘The World to Come’ y ‘Ammonite’ insisten en el interés actual por las historias lésbicas ambientadas e…
6,2306,0.939,Reforma,@SCJN empezará a discutir mañana un proyecto que declara la validez del Acuerdo Presidencial de mayo de 2020 por el cual las FA seguirán realizando labores de seguridad pública por lo menos hasta marzo de 2024 https://t.co/zsRvc69QOJ
7,7881,0.939,el_pais,"El festival de San Sebastián se defiende: ""No ha sido acusado o condenado por ninguna forma de agresión o violencia contra ninguna mujer"". Pero un juez determinó que Johnny Depp agredió a su exesposa, Amber Heard, en 12 ocasiones de las 14 aludidas https://t.co/JirGrh899y"


  0.001365 seconds (4 allocations: 96 bytes)


1,13276,0.938,bbcmundo,EE.UU. ha dejado atrás una fuerza afgana mucho menos dotada y capacitada para repeler el avance del Talibán https://t.co/fEHroPHcm2
2,9839,0.942,PublimetroMX,"Amber Heard declaró que Depp la agredió física y sexualmente en múltiples ocasiones, generalmente cuando supuestamente estaba borracho o drogado. https://t.co/B5JpHfI3o6"
3,5348,0.944,CNNEE,"""I Am Legend"" es pura fantasía, aclaró Akiva Goldsman, quien coescribió el guión de la película de 2007. Esto después de que una fuente anónima le dijera a The New York Times que le temía a la vacuna contra el covid-19 debido a lo que pasó en la película. https://t.co/S23wIQUfhy"
4,10627,0.944,CNNEE,El jefe de Inteligencia del Reino Unido afirma que los soldados rusos tienen la moral baja y se niegan a cumplir órdenes. https://t.co/73JQEPfTu2 https://t.co/TlbhZ3d8Pe
5,25347,0.944,el_pais,RT @elpais_valencia: Un concejal valenciano cuenta en un pleno que es homosexual: “No por ser de Vox tengo que ser heterosexual” https://t.…
6,25599,0.946,bbcmundo,"Además, hay que aceptar un diagnóstico psiquiátrico de ""trastorno de identidad de género"", entre otros requisitos. https://t.co/fZfpbam1JD"
7,15661,0.946,Reforma,"Los críticos de @laveoonolaveo te hablan de la comedia ""Free Guy: Tomando el Control"", protagonizada por Ryan Reynolds. https://t.co/i2CwQWPdfV https://t.co/i2ZAPOQjQa"


  0.001769 seconds (4 allocations: 96 bytes)


1,9839,0.916,PublimetroMX,"Amber Heard declaró que Depp la agredió física y sexualmente en múltiples ocasiones, generalmente cuando supuestamente estaba borracho o drogado. https://t.co/B5JpHfI3o6"
2,14241,0.921,UniNoticias,"🇷🇺 Donbas, el polvorín del este de Ucrania que puede servir de excusa para una invasión rusa. 🇺🇦 En esa zona hay un enfrentamiento entre grupos separatistas prorrusos y fuerzas ucranianas. https://t.co/90nkyG70L9"
3,11594,0.922,EFEnoticias,"Un grupo de periodistas de investigación ucranianos asegura haber podido identificar a algunos de los presuntos autores de disparos contra civiles en la localidad de Bucha, como pertenecientes a una brigada naval del Ejército ruso del Pacífico. https://t.co/pKq8QVAnfp"
4,520,0.922,bbcmundo,"Las autoridades ucranianas han descubierto evidencias dejadas por las tropas rusas en Bucha, donde un campo de verano para niños terminó convertido en un centro de ejecución de civiles. https://t.co/cwJuqwOFLt"
5,19157,0.923,TelemundoNews,"El candidato trabaja como investigador privado y el día del incidente estaba cerca de una casa rentada por el actual alcalde de Miami, Francis Suarez. Cuando le preguntaron que hacía ahí, dijo que estaba con la oficina del Sheriff. https://t.co/01CxpZ1xvh"
6,19932,0.923,abc_es,Una juez investiga por coacciones y estafa a una empresa que cobraba deudas de eléctricas https://t.co/oamchYZHSt
7,4280,0.925,PublimetroChile,"El superintendente de Educación, indicó que la Ley de Aula Segura no se utilizó, ya que empleó el reglamento interno del Colegio Verbo Divino https://t.co/MjT4JRltz3"


In [15]:
display(@htl "<h1>Ejemplos de búsqueda (mensajes aleatorios sobre el test)</h1>")

for i in 1:3
    for qid in rand(1:length(Q.corpus))
        search_and_display(E, Q.corpus[qid], 7)
    end
end

<h1>Ejemplos de búsqueda (mensajes aleatorios sobre el test)</h1>

  0.001407 seconds (4 allocations: 96 bytes)


1,6320,0.915,CNNEE,"Durante media década, la franquicia de terror ""Viernes 13"" ha estado atrapada en una disputa de derechos de autor sobre quién es el propietario del guion original. ¿Te gustan estas películas? https://t.co/noTzrPtjni"
2,12515,0.918,el_pais,RT @elpais_america: Los estrenos de ‘The World to Come’ y ‘Ammonite’ insisten en el interés actual por las historias lésbicas ambientadas e…
3,16684,0.92,PublimetroCol,Elden Ring presentará gameplay el día de mañana https://t.co/SeS9oHtyvK
4,2537,0.923,Radio_Formula,¿Te gustaría verla de nuevo en un melodrama? https://t.co/C2OwQY6Qie
5,26628,0.924,bbcmundo,"Aunque los rusos se han enfrentado a una fuerte resistencia de las fuerzas ucranianas, tienen ventajas cruciales en el campo de batalla. https://t.co/WRoUwlUvp2"
6,17799,0.924,EFEnoticias,En España se realizan anualmente cerca de 100.000 abortos voluntarios y el Ministerio de Igualdad sitúa entre sus prioridades reformar la ley que los ampara. https://t.co/WdNMqI2kyD
7,9163,0.924,UniNoticias,El gobernador de Texas confirma que ese estado construirá su propio muro en la frontera con México. https://t.co/l3FuUc4USO


  0.001614 seconds (4 allocations: 96 bytes)


1,14241,0.907,UniNoticias,"🇷🇺 Donbas, el polvorín del este de Ucrania que puede servir de excusa para una invasión rusa. 🇺🇦 En esa zona hay un enfrentamiento entre grupos separatistas prorrusos y fuerzas ucranianas. https://t.co/90nkyG70L9"
2,9163,0.91,UniNoticias,El gobernador de Texas confirma que ese estado construirá su propio muro en la frontera con México. https://t.co/l3FuUc4USO
3,520,0.911,bbcmundo,"Las autoridades ucranianas han descubierto evidencias dejadas por las tropas rusas en Bucha, donde un campo de verano para niños terminó convertido en un centro de ejecución de civiles. https://t.co/cwJuqwOFLt"
4,7077,0.912,bbcmundo,"El reconocimiento de las ""repúblicas independientes"" de Donetsk y Luhansk por parte de Rusia ha llevado al conflicto entre Ucrania 🇺🇦 y Rusia 🇷🇺 al siguiente nivel. Pero, ¿por qué Rusia reconoce la independencia de estos territorios? https://t.co/OrxXx89EVm"
5,9839,0.914,PublimetroMX,"Amber Heard declaró que Depp la agredió física y sexualmente en múltiples ocasiones, generalmente cuando supuestamente estaba borracho o drogado. https://t.co/B5JpHfI3o6"
6,16554,0.917,TelemundoNews,La policía detiene al sospechoso del tiroteo en un salón de belleza de Dallas. El FBI abre investigación por delito de odio. https://t.co/WEkzuEccLx
7,25809,0.918,CNNEE,En la capital de Ucrania suenan las sirenas de ataques aéreos y se levantan barreras antitanques https://t.co/8VIQ0GtR4z


  0.001783 seconds (4 allocations: 96 bytes)


1,14241,0.905,UniNoticias,"🇷🇺 Donbas, el polvorín del este de Ucrania que puede servir de excusa para una invasión rusa. 🇺🇦 En esa zona hay un enfrentamiento entre grupos separatistas prorrusos y fuerzas ucranianas. https://t.co/90nkyG70L9"
2,14074,0.906,PublimetroChile,Al menos 14 estados brasileños declararon estado de emergencia👇 https://t.co/ZnEQleVzTB
3,11771,0.906,el_pais,"Sí, todo el mundo sabe utilizar Google. Pero, ¿sabemos realmente sacar todo el potencial de este buscador? https://t.co/mNDFaiMwQD"
4,8083,0.907,CNNEE,¿Eres de teflón o de velcro frente al estrés? La respuesta es importante. La psiquiatra Samantha Boardman nos explica por qué. https://t.co/vgNKhh4DCz
5,8059,0.908,TelemundoNews,La FDA investiga por qué tardaron en inspeccionar la posible contaminación en una planta de fórmula para bebés. https://t.co/wHpjN0oQv5
6,2537,0.909,Radio_Formula,¿Te gustaría verla de nuevo en un melodrama? https://t.co/C2OwQY6Qie
7,24624,0.909,UniNoticias,🇷🇺 🇺🇦 Rusia dice que su ejército bloquea dos ciudades costeras y una base aérea en el sur de Ucrania. https://t.co/7tPKpChniu


# Actividades
- Reproduzca el ejercicio de este notebook, use embeddings para español, cambié los ejemplos. Se sugiere el uso de <https://ingeotec.github.io/regional-spanish-models/> donde encontrará modelos fastText regionalizados del español, pero puede usar otros embeddings.
- ¿Qué piensa de las diferencias de tamaño entre los documentos y las consultas? esto como afecta a la representación semántica.
- ¿Cuál sería el símil de bigramas y trigramas para este esquema de representación semántica? Implementelo.
- Implemente su búsqueda semántica. Si usa Julia considere `SimilaritySearch.jl` y si usa Python considere `faiss`.
- Reporte su notebook y anote sus soluciones a las preguntas planteadas. Reporte los resultados de sus implementaciones, compare contra las alternativas presentadas en este reporte. Discuta sus resultados. Finalice el reporte  con reflexiones sobre el uso de nubes de puntos en lugar de bolsas de palabras tradicionales. Anoté sus conclusiones.

# Bibliografía
- [KSKW2015] Kusner, M., Sun, Y., Kolkin, N., & Weinberger, K. (2015, June). From word embeddings to document distances. In International conference on machine learning (pp. 957-966). PMLR.
- [PW2009] Pele, O., & Werman, M. (2009, September). Fast and robust earth mover's distances. In 2009 IEEE 12th international conference on computer vision (pp. 460-467). IEEE.
- [VSPU2017] Vaswani, A., Shazeer, N., Parmar, N., Uszkoreit, J., Jones, L., Gomez, A. N., ... & Polosukhin, I. (2017). Attention is all you need. Advances in neural information processing systems, 30.
